In [34]:
import pandas as pd

## Задание 1

In [198]:
nams = ('surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome')

In [199]:
horses_df = pd.read_csv('horse_data.csv', na_values = '?', header = None).iloc[:, [0,1,3,4,5,6,11,22]]
horses_df.columns = nams

In [200]:
horses_df

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,4.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,4.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,4.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...
295,1.0,1,NaN,120.0,70.0,4.0,4.0,3.0
296,2.0,1,37.2,72.0,24.0,3.0,3.0,3.0
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


## Задание 2

In [201]:
horses_df.describe()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,244.000000,256.000000,299.000000
mean,1.397993,1.640000,38.167917,71.913043,30.417355,2.348361,2.917969,1.551839
std,0.490305,2.173972,0.732289,28.630557,17.642231,1.045054,0.976744,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,3.000000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.000000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.000000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,4.000000,3.000000


Посчитаем выбросы для столбцов с количественными показателями без учета не заполненных строк (NaN)

In [202]:
def blowout (heder, result_output = True):
    q1 = horses_df[heder].quantile(0.25)
    q3 = horses_df[heder].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr) 
    upper_bound = q3 + (1.5 * iqr)
    remove_outliers = horses_df[horses_df[heder].between(lower_bound, upper_bound, inclusive=True)]
    if result_output == True:
        return pd.concat([horses_df, remove_outliers]).drop_duplicates(keep=False).sort_values(heder, ascending = False).dropna(subset=[heder])
    elif result_output == False:
        result = pd.concat([horses_df, remove_outliers]).drop_duplicates(keep=False).dropna(subset=[heder]).shape[0]
        print(f'Число выбросов в столбце {heder} - {result}')

In [203]:
blowout('rectal temperature', result_output = False)
blowout('pulse', result_output = False)
blowout('respiratory rate', result_output = False)

Число выбросов в столбце rectal temperature - 14
Число выбросов в столбце pulse - 5
Число выбросов в столбце respiratory rate - 15


## Задание 3

In [204]:
horses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    299 non-null    float64
 1   Age                         300 non-null    int64  
 2   rectal temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory rate            242 non-null    float64
 5   temperature of extremities  244 non-null    float64
 6   pain                        256 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


**surgery?** - пропущено 1 значение,  
**Age** - может принимать значения 1 или 2, найден максимум - 9 это значит, что в данных есть как минимум 1 ошибка, нужно проверить подробнее    
**rectal temperature** - пропущено 60 значений, это довольно много, но средняя температура 38,2 близка к нормальной 37,8 СКО не велико, пропуски можем заменить средними или медианой  
**pulse** - 24 пропуска. Пульс у лошадей зависит от возраста и физической нагрузки, минимальный возможный - 30, максимальный 240 (быстрый поиск в гугле). СКО довольно высоко, можно было бы попробовать сгруппировать по возрасту и заменить пропуски медианными значениями  
**respiratory rate** - много пропусков, в описании говориться, что показатель малополезен из-за больших колебаний, высокое СКО подтверждает большой разброс показателей    
**temperature of extremities** - много пропусков, показатель качественный номинальный  
**pain** - много пропусков, показатель качественный порядковый  
**outcome** - 1 пропуск

In [226]:
# показатель качественный номинальный, можем заменить пропуски на 'no_info'
horses_df['surgery?'].fillna('no_info', inplace=True)
horses_df['surgery?'].isna().sum()

0

In [227]:
# проверяем количество ошибок
horses_df.value_counts('Age')

Age
1    276
9     24
dtype: int64

24 ошибки в данных по столбцу **Age**

In [228]:
# как и в пердыдущем столбце меняем ошибки на 'no_info'
horses_df.loc[horses_df['Age'] == 9, 'Age'] = 'no_info'
horses_df['Age'].isna().sum()

0

In [229]:
# заменяем пропуски средним
horses_df['rectal temperature'].fillna(horses_df['rectal temperature'].mean(), inplace=True)
horses_df['rectal temperature'].isna().sum()

0

In [230]:
#группируем по возрасту (делаем это в учебных целях, все лошади старше 0.5 лет и группировка ничего не изменит) заменяем пропуски медианой
horses_df['pulse'].fillna(horses_df.groupby('Age')['pulse'].transform('median'), inplace=True)
horses_df['pulse'].isna().sum()

0

In [231]:
#заменяем пропуски медианой
horses_df['respiratory rate'].fillna(horses_df['respiratory rate'].median(), inplace=True)
horses_df['respiratory rate'].isna().sum()

0

In [232]:
# показатель качественный номинальный, заменяем пропуски на 'no_info'
horses_df['temperature of extremities'].fillna('no_info', inplace=True)
horses_df['temperature of extremities'].isna().sum()

0

In [233]:
# посмотрим сколько было проведено операций с разными степенями боли
horses_df.pivot_table(index = 'pain', columns = 'surgery?', values = 'Age', aggfunc = 'count', margins = True)

surgery?,1.0,2.0,no_info,All
pain,,,,
1.0,11.0,28.0,NaN,39
2.0,11.0,5.0,NaN,16
3.0,78.0,49.0,1.0,128
4.0,55.0,18.0,NaN,73
no_info,25.0,19.0,NaN,44
All,180.0,119.0,1.0,300


In [234]:
# в целом операций было меньше чем лечений без хирургического вмешательства только при отсутствии боли (1), 
# какой-то выраженной закономерности не наблюдается, значит заменим пропуски на 'no_info'
horses_df['pain'].fillna('no_info', inplace=True)
horses_df['pain'].isna().sum()

0

In [235]:
# 1 пропущенное значение в outcome востановить на основе имеющихся данных так же не можем, меняем на 'no_info'
horses_df['outcome'].fillna('no_info', inplace=True)
horses_df['outcome'].isna().sum()

0

In [236]:
horses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    300 non-null    object 
 1   Age                         300 non-null    object 
 2   rectal temperature          300 non-null    float64
 3   pulse                       300 non-null    float64
 4   respiratory rate            300 non-null    float64
 5   temperature of extremities  300 non-null    object 
 6   pain                        300 non-null    object 
 7   outcome                     300 non-null    object 
dtypes: float64(3), object(5)
memory usage: 18.9+ KB
